# <p style="text-align: left;"> <span style="font-family:consolas">Deep Learning _ Final Project on <font color = purple > TRANSFER LEARNING  (PART 1)</span>  </p>
***

> ## Members: 
        > ### <font color = blue > <span style="font-family:consolas"> Amanuel Abrdo Tereda </font>
        > ### <font color = blue > <span style="font-family:consolas"> Stefalo Acha </font>
***
> *Date: May 02, 2023*
> ### <p style="text-align: right;">Instructor: <font color = blue > Dr. Letu Qingge</p>
***



## Image classification with different Neural Netowrk setups

> **1. 100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss**

> **2.	100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss**

> **3.	100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss**

> **4.	100 Sigmoid hidden units, 5 Linear output units, Softmax loss**

> **5.	100 Tanh hidden units, 5 Linear output units, Softmax loss**

        - Without an optimizer and an initialization
    
        - With an optimizer of momentum = 0.9, weight decay = 0.02, and 'glorot' initialization.

        - With an optimizer of momentum = 0.9, weight decay = 0.02, and 'random' initialization.

***

In [1]:
!git clone https://github.com/Amumu-ze1ast/Deep_Learning_Final_try.git

Cloning into 'Deep_Learning_Final_try'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 31.09 KiB | 2.59 MiB/s, done.


In [ ]:
import zipfile
with zipfile.ZipFile("/content/Deep_Learning_Final_try/lincoln.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import lincoln
from lincoln.layers import Dense
from lincoln.losses import SoftmaxCrossEntropy, MeanSquaredError
from lincoln.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.network import NeuralNetwork
from lincoln.train import Trainer
from lincoln.utils import mnist
from lincoln.utils.np_utils import softmax

In [ ]:
import numpy as np
import cv2
import pandas as pd
import PIL.Image as Image
import os
import matplotlib.pylab as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
list_images = list(data_dir.glob('*/*.jpg'))
image_count = len(list_images)

In [ ]:
#Make up python dictionary

flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

#Creating a label directory

flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

class_names = list(flowers_labels_dict.keys())

In [ ]:
# Resize all images using for loop

IMAGE_SHAPE = (224,224)
#IMAGE_SHAPE = (28,28)

x, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(img_gray,(IMAGE_SHAPE)) #IMAGE_SHAPE = (224,224)
        
        #resized_img = cv2.resize(img,(IMAGE_SHAPE)) #IMAGE_SHAPE = (224,224)
        
        x.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [ ]:
x[0].shape

(224, 224)

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3, random_state=25)

In [ ]:
x_train, x_test = x_train - np.mean(x_train), x_test - np.mean(x_train)

In [ ]:
np.min(x_train), np.max(x_train), np.min(x_test), np.max(x_test)

(-107.73345109759843,
 147.26654890240155,
 -107.73345109759843,
 147.26654890240155)

In [ ]:
x_train, x_test = x_train / np.std(x_train), x_test / np.std(x_train)

In [ ]:
np.min(x_train), np.max(x_train), np.min(x_test), np.max(x_test)

(-1.626436750452479,
 2.2232623647244543,
 -1.626436750452479,
 2.2232623647244543)

In [ ]:
#verify
print("shape of input  - training set =", x_train.shape)
print("shape of output - training set =", y_train.shape)
print("shape of input  - testing set  =", x_test.shape)
print("shape of output - testing set  =", y_test.shape)

shape of input  - training set = (2446, 224, 224)
shape of output - training set = (2446,)
shape of input  - testing set  = (1224, 224, 224)
shape of output - testing set  = (1224,)


In [ ]:
x_train.shape[1]

224

In [ ]:
x_train = x_train.reshape(len(x_train),(x_train.shape[1])*(x_train.shape[1]))
x_test  = x_test.reshape(len(x_test),(x_test.shape[1])*(x_test.shape[1]))

In [ ]:
#verify
print("shape of input  - training set =", x_train.shape)
print("shape of output - training set =", y_train.shape)
print("shape of input  - testing set  =", x_test.shape)
print("shape of output - testing set  =", y_test.shape)

shape of input  - training set = (2446, 50176)
shape of output - training set = (2446,)
shape of input  - testing set  = (1224, 50176)
shape of output - testing set  = (1224,)


In [ ]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 5))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 5))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [ ]:
#verify
print("shape of input  - training set    =", x_train.shape)
print("shape of output - training lables =", train_labels.shape)
print("shape of input  - testing set     =", x_test.shape)
print("shape of output - test labels     =", test_labels.shape)

shape of input  - training set    = (2446, 50176)
shape of output - training lables = (2446, 5)
shape of input  - testing set     = (1224, 50176)
shape of output - test labels     = (1224, 5)


In [ ]:
def calc_accuracy_model(model, test_set):
    return print(f'''The model validation accuracy is: 
    {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

### a) 100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss

In [ ]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=ReLU()),
            Dense(neurons=5, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

print('100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss
_________________________________/\____________________________________
Validation loss after 10 epochs is 1.241
Validation loss after 20 epochs is 1.093
Validation loss after 30 epochs is 1.028

Loss increased after epoch 40, final loss was 1.028, 
using the model from epoch 30

_________________________________\/____________________________________
The model validation accuracy is: 
    19.85%


### b) 100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss 

In [ ]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh()),
            Dense(neurons=5, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

print('100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss
_________________________________/\____________________________________
Validation loss after 10 epochs is 1.800
Validation loss after 20 epochs is 1.661
Validation loss after 30 epochs is 1.591
Validation loss after 40 epochs is 1.537
Validation loss after 50 epochs is 1.491
Validation loss after 60 epochs is 1.465
Validation loss after 70 epochs is 1.441

_________________________________\/____________________________________
The model validation accuracy is: 
    25.98%


### c) 100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss

In [ ]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Sigmoid()),
            Dense(neurons=5, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

print('100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss
_________________________________/\____________________________________
Validation loss after 10 epochs is 1.043
Validation loss after 20 epochs is 1.021

Loss increased after epoch 30, final loss was 1.021, 
using the model from epoch 20

_________________________________\/____________________________________
The model validation accuracy is: 
    22.71%


### d) 100 Sigmoid hidden units, 5 Linear output units, Softmax loss 

In [ ]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Sigmoid()),
            Dense(neurons=5, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

print('100 Sigmoid hidden units, 5 Linear output units, Softmax loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 10,
            eval_every = 1,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Sigmoid hidden units, 5 Linear output units, Softmax loss
_________________________________/\____________________________________
Validation loss after 1 epochs is 6.762
Validation loss after 2 epochs is 6.318
Validation loss after 3 epochs is 5.919
Validation loss after 4 epochs is 5.645
Validation loss after 5 epochs is 5.480
Validation loss after 6 epochs is 5.341
Validation loss after 7 epochs is 5.197
Validation loss after 8 epochs is 5.055
Validation loss after 9 epochs is 4.901
Validation loss after 10 epochs is 4.764

_________________________________\/____________________________________
The model validation accuracy is: 
    24.59%


### e) 100 Tanh hidden units, 5 Linear output units, Softmax loss

In [ ]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh()),
            Dense(neurons=5, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

print('100 Tanh hidden units, 5 Linear output units, Softmax loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Linear output units, Softmax loss
_________________________________/\____________________________________
Validation loss after 10 epochs is 5.058
Validation loss after 20 epochs is 3.107
Validation loss after 30 epochs is 2.781
Validation loss after 40 epochs is 2.755
Validation loss after 50 epochs is 2.736
Validation loss after 60 epochs is 2.727

Loss increased after epoch 70, final loss was 2.727, 
using the model from epoch 60

_________________________________\/____________________________________
The model validation accuracy is: 
    27.12%


## Comparing the weight initialization algorithms for the best model

### a) Random

In [ ]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh(),
                  weight_init="random"),
            Dense(neurons=5, 
                  activation=Linear(),
                  weight_init="random")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.25, 
                        momentum=0.9, 
                        final_lr = 0.02, 
                        decay_type='exponential')

print('100 Tanh hidden units, 5 Linear output units, Softmax loss')
print("With an optimizer of momentum = 0.9, weight decay = 0.02, and 'random' initialization")
print('_________________________________/\____________________________________')

trainer = Trainer(model, optimizer)
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Linear output units, Softmax loss
With an optimizer of momentum = 0.9, weight decay = 0.02, and 'random' initialization
_________________________________/\____________________________________
Validation loss after 10 epochs is 4.128
Validation loss after 20 epochs is 3.265
Validation loss after 30 epochs is 3.024
Validation loss after 40 epochs is 2.894
Validation loss after 50 epochs is 2.777

Loss increased after epoch 60, final loss was 2.777, 
using the model from epoch 50

The model validation accuracy is: 
    29.17%


### b) Glorot 

In [ ]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=5, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.25, 
                        momentum=0.9, 
                        final_lr = 0.02, 
                        decay_type='exponential')

print('100 Tanh hidden units, 5 Linear output units, Softmax loss')
print("With an optimizer of momentum = 0.9, weight decay = 0.02, and 'glorot' initialization")
print('_________________________________/\____________________________________')

trainer = Trainer(model, optimizer)
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Linear output units, Softmax loss
With an optimizer of momentum = 0.9, weight decay = 0.02, and 'glorot' initialization
_________________________________/\____________________________________
Validation loss after 10 epochs is 4.040
Validation loss after 20 epochs is 3.296

Loss increased after epoch 30, final loss was 3.296, 
using the model from epoch 20

_________________________________\/____________________________________
The model validation accuracy is: 
    27.29%
